In [16]:
import pandas as pd

In [ ]:
data = pd.read_csv('../data/01_raw/data.csv')
data

In [7]:
separador = '2PT Field Goal'
filter_2fg_partial = data[data['shot_type'] == separador]
filter_3fg_partial = data[data['shot_type'] != separador]

filter_2fg = filter_2fg_partial[['lat',
                                     'lon',
                                     'minutes_remaining',
                                     'period',
                                     'playoffs',
                                     'shot_distance',
                                     'shot_made_flag']]
filter_3fg = filter_3fg_partial[['lat',
                                     'lon',
                                     'minutes_remaining',
                                     'period',
                                     'playoffs',
                                     'shot_distance',
                                     'shot_made_flag']]

filter_2fg = filter_2fg.dropna()
filter_3fg = filter_3fg.dropna()

In [ ]:
filter_2fg

In [ ]:
X = filter_2fg.loc[:, filter_2fg.columns != 'shot_made_flag']
X

In [ ]:
y = filter_2fg.loc[:, filter_2fg.columns == 'shot_made_flag']
y

In [ ]:
filter_3fg = filter_3fg.dropna()
filter_3fg

In [21]:
def metrics(X_test, X_train):
    metrics = {
        'X_test_rows': {'value':X_test.shape[0], 'step':1},
        'X_test_cols': {'value':X_test.shape[1], 'step':2},
        'X_train_rows': {'value':X_train.shape[0], 'step':3},
        'X_train_cols': {'value':X_train.shape[1], 'step':4}
    }
    print(metrics)

    return {
       'X_test_rows': {'value':X_test.shape[0], 'step':1},
        'X_test_cols': {'value':X_test.shape[1], 'step':2},
        'X_train_rows': {'value':X_train.shape[0], 'step':3},
        'X_train_cols': {'value':X_train.shape[1], 'step':4}
    }

X_test = pd.read_parquet('../data/04_feature/X_test_data.parquet')
X_train = pd.read_parquet('../data/04_feature/X_train_data.parquet')
y_test = pd.read_parquet('../data/04_feature/y_test_data.parquet')
y_train = pd.read_parquet('../data/04_feature/y_train_data.parquet')

retorno = metrics(X_test,X_train)
print(retorno)

{'X_test_rows': {'value': 4057, 'step': 1}, 'X_test_cols': {'value': 6, 'step': 2}, 'X_train_rows': {'value': 16228, 'step': 3}, 'X_train_cols': {'value': 6, 'step': 4}}
{'X_test_rows': {'value': 4057, 'step': 1}, 'X_test_cols': {'value': 6, 'step': 2}, 'X_train_rows': {'value': 16228, 'step': 3}, 'X_train_cols': {'value': 6, 'step': 4}}
